In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [0]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [0]:
new_lines_neu = []
categories_neu = {}
categories_neg = {}
categories_pos = {}

In [0]:
with open('TA_NEU.txt','r') as f:
  lines_neu = f.readlines()
  lines_neu = lines_neu[65:]
  for line in lines_neu:
      new_lines = line.split()
      categories_neu[new_lines[1]] = [0,0,1]


In [0]:
with open('TA_NEG.txt','r') as f:
  lines_neg = f.readlines()
  lines_neg = lines_neg[65:]
  for line in lines_neg:
      new_lines = line.split()
      categories_neg[new_lines[1]] = [0,1,0]

In [0]:
with open('TA_POS.txt','r') as f:
  lines_pos = f.readlines()
  lines_pos = lines_pos[65:]
  for line in lines_pos:
      new_lines = line.split()
      categories_pos[new_lines[1]] = [1,0,0]

In [0]:
merge = [categories_neg,categories_neu,categories_pos]
new_categories = {}
for x in merge:
  new_categories.update(x)

In [0]:
keys = list(new_categories.keys())
random.shuffle(keys)
new_categories1 = [(key, new_categories[key]) for key in keys]

In [0]:
new_categories1 = dict(new_categories1)
len(new_categories1)

In [0]:
categories = []

In [0]:
keys = list(new_categories1.keys())
for i,k in new_categories1.items():
  categories.append(k)


In [0]:
len(categories)

In [0]:
all_letters = set('அஇஉஎஒஆஈஊஏஓஐஔங்ங்ஞ்ண்ந்ம்ன்க்ச்ட்த்ப்ற்ய்ர்ல்வ்ழ்ள்ஃ')
all_letters = list(all_letters)
len(all_letters)
n_letters = 32

In [0]:
class SelfAttention(nn.Module):
  def __init__(self,bacth_size,output_size,hidden_size,vocab_size,embedding_length):
    super(SelfAttention,self).__init__()
    self.batch_size = batch_size
    self.output_size = output_size
    self.hidden_size = hidden_size
    self.vocab_size = vocab_size
    self.embedding_length = embedding_length
    
    self.word_embeddings = nn.Embedding(vocab_size,embedding_length)

    self.bilstm = nn.LSTM(embedding_length, hidden_size, bidirectional=True)
    self.W_s1 = nn.Linear(2*hidden_size, 350)
    self.W_s2 = nn.Linear(350, 30)
    self.fc_layer = nn.Linear(30*2*hidden_size, 2000)
    self.label = nn.Linear(2000, output_size)
  def attention_net(self,lstm_output):
    attn_weight_matrix = self.W_s2(F.tanh(self.W_s1(lstm_output)))
    attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1)
    attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)
    return attn_weight_matrix
  def forward(self, input_sentences, batch_size=1):
    input_sentences = input_sentences.long()
    inputs = self.word_embeddings(input_sentences)
    inputs = inputs.squeeze(2)
    h_0 = torch.zeros(2, 32, self.hidden_size).cuda()
    c_0 = torch.zeros(2, 32, self.hidden_size).cuda()
    output, (h_n, c_n) = self.bilstm(inputs, (h_0, c_0))
    output = output.permute(1, 0, 2)
    attn_weight_matrix = self.attention_net(output)
    hidden_matrix = torch.bmm(attn_weight_matrix, output)
    fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
    logits = self.label(fc_out)
    return logits


In [0]:

import torch

def letterToIndex(letter):
  index = -1
  try:
    for i,k in enumerate(all_letters):
      if letter == k:
        return i
  except:
    print('please enter a valid character')

def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        if letterToIndex(letter) != -1:
          tensor[li][0][letterToIndex(letter)] = 1
        
    return tensor
    

In [0]:
vocab_size = n_letters
embedding = n_letters
output_size = 3
batch_size = 1
hidden_size = 512


attention = SelfAttention(batch_size, output_size, hidden_size, vocab_size,embedding)

In [0]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    l = [0,0,0]
    l[category_i] = l[category_i]+1
    return l

In [0]:
import torch.optim as optim
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params = attention.parameters(),lr = 0.5)
 
def train(category_tensors, line_tensors):
  final_loss = 0
  correct_count = 0
  for i in range(0,100):
    line_tensor = line_tensors[i]
    category = category_tensors[i] 
    category_tensor = torch.tensor(category, dtype=torch.float)
   
    
    for j in range(line_tensor.size()[0]):
        output = attention(line_tensor[j].unsqueeze(2))
    guess_i = categoryFromOutput(output)
    
    if guess_i == category:
      correct =  '✓'
      correct_count = correct_count + 1
    else:
      correct =  '✗ (%s)' % category 
    loss = criterion(output, category_tensor.view(1,3))
    final_loss = final_loss + loss

  final_loss = final_loss/100

  final_loss.backward()
  optimizer.step()

  return final_loss,correct_count   

In [0]:
n_iters = 4000
cat_tensors = []
line_tensors = []
all_losses = []
epochs = 5
for epoch in range(epochs):

  batch_num = 0
  correct_sum = 0
  final_loss = 0
  attention.zero_grad()
  optimizer.zero_grad()

  for iter in range(0,n_iters+1):
    
    while True:
      
      category = new_categories1[keys[iter]]
      cat_tensors.append(category)
      line_tensor = lineToTensor(keys[iter])
      line_tensors.append(line_tensor)
      if iter % 100 != 0 or iter == 0:
        break
      else:
        batch_num = batch_num + 1
        final_loss,correct = train(cat_tensors,line_tensors)
        correct_sum = correct_sum + correct
        cat_tensors.clear()
        line_tensors.clear()
        if iter != 0:
          print('%d %s %.4f  %s %s' % (batch_num, iter , final_loss, correct_sum,epoch))
          break
  print( 'accuracy is %.6f%%' % (correct_sum/4000 * 100))    